In [16]:
import tensorflow as tf
import numpy as np

In [19]:
height = 28
width = 28
channels = 1
n_inputs = height * width

conv1_fmaps = 32
conv1_ksize = 3
conv1_stride = 1
conv1_pad = 'SAME'

conv2_fmaps = 64
conv2_ksize = 3
conv2_stride = 2
conv2_pad = 'SAME'

pool3_fmaps = conv2_fmaps

n_fc1 = 64
n_outputs = 10

tf.reset_default_graph()

In [20]:
with tf.name_scope('inputs'):
    X = tf.placeholder(name='X', dtype=tf.float32, shape=(None, n_inputs))
    X_reshaped = tf.reshape(X, shape=(-1, height, width, channels))
    y = tf.placeholder(name='y', dtype=tf.int32, shape=[None])
print(X_reshaped.shape)
conv1 = tf.layers.conv2d(X_reshaped, filters=conv1_fmaps, kernel_size=conv1_ksize, strides=conv1_stride,
                         padding=conv1_pad, activation=tf.nn.elu, name='conv1')
print(conv1.shape)
conv2 = tf.layers.conv2d(conv1, filters=conv2_fmaps, kernel_size=conv2_ksize, strides=conv2_stride, 
                        padding=conv2_pad, activation=tf.nn.elu, name='conv2')

print(conv2.shape)
with tf.name_scope('pool3'):
    pool3 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    pool3_flat = tf.reshape(pool3, shape=[-1, pool3_fmaps * 7 * 7])
print(pool3.shape)
print(pool3_flat.shape)
with tf.name_scope('fc1'):
    fc1 = tf.layers.dense(pool3_flat, n_fc1, activation=tf.nn.elu, name='fc1')
    
with tf.name_scope('output'):
    logits = tf.layers.dense(fc1, n_outputs, name='output')
    y_prob = tf.nn.softmax(logits, name='y_prob')
    
with tf.name_scope('train'):
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(cross_entropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)
    
with tf.name_scope('eval'):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))
    
with tf.name_scope('init_and_save'):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()

(?, 28, 28, 1)
(?, 28, 28, 32)
(?, 14, 14, 64)
(?, 7, 7, 64)
(?, 3136)


In [6]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('/tmp/data/')

Extracting /tmp/data/train-images-idx3-ubyte.gz
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [15]:
type(mnist.train)

tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet

In [12]:
n_epochs = 10
batch_size = 100

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(mnist.train.num_examples // batch_size):
            X_batch, y_batch = mnist.train.next_batch(batch_size=batch_size)
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_val = accuracy.eval(feed_dict={X: mnist.validation.images, y: mnist.validation.labels})
        print('epoch: {}, training accuracy: {}, validation accuracy: {}'.format(epoch, acc_train, acc_val))
        
        saver_path = saver.save(sess, '/tmp/mnist_with_cnn1.ckpt')

epoch: 0, training accuracy: 0.949999988079071, validation accuracy: 0.9805999994277954
epoch: 1, training accuracy: 1.0, validation accuracy: 0.982200026512146
epoch: 2, training accuracy: 0.9800000190734863, validation accuracy: 0.9860000014305115
epoch: 3, training accuracy: 0.9900000095367432, validation accuracy: 0.9860000014305115
epoch: 4, training accuracy: 1.0, validation accuracy: 0.9879999756813049
epoch: 5, training accuracy: 1.0, validation accuracy: 0.9873999953269958
epoch: 6, training accuracy: 1.0, validation accuracy: 0.9876000285148621
epoch: 7, training accuracy: 1.0, validation accuracy: 0.984000027179718
epoch: 8, training accuracy: 1.0, validation accuracy: 0.9883999824523926
epoch: 9, training accuracy: 1.0, validation accuracy: 0.9891999959945679


In [13]:
with tf.Session() as sess:
    saver.restore(sess, '/tmp/mnist_with_cnn1.ckpt')
    Z = logits.eval(feed_dict= {X: mnist.test.images})
    y_pred = np.argmax(Z, axis=1)
    print(np.mean(y_pred == mnist.test.labels).astype(np.float32))

INFO:tensorflow:Restoring parameters from /tmp/mnist_with_cnn1.ckpt
0.987
